# 3D Magnetic Field Diffusion (Rm << 1)

**Validation case**: Magnetic field diffusion in the resistive limit

## Learning Objectives

After completing this notebook, you will understand:

1. The magnetic induction equation and the role of resistivity
2. The magnetic Reynolds number and what Rm << 1 means physically
3. How a Gaussian magnetic field profile spreads due to diffusion
4. The analytic solution for magnetic diffusion in 2D and 3D
5. How ResistiveMHD and ExtendedMHD models compare for pure diffusion

---
## 1. Physics Background

### The Magnetic Induction Equation

The evolution of magnetic field in a conducting plasma is governed by the **induction equation**:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

where:
- $\mathbf{v}$ is the plasma velocity
- $\eta = 1/(\mu_0 \sigma)$ is the magnetic diffusivity (related to resistivity)
- $\sigma$ is the electrical conductivity

The two terms represent competing physics:

| Term | Physics | Effect |
|------|---------|--------|
| $\nabla \times (\mathbf{v} \times \mathbf{B})$ | Advection | Field frozen to plasma, moves with flow |
| $\eta \nabla^2 \mathbf{B}$ | Diffusion | Field slips through plasma, spreads out |

### The Magnetic Reynolds Number

The **magnetic Reynolds number** measures the relative importance of advection vs diffusion:

$$R_m = \frac{v L}{\eta} = \frac{\text{advection rate}}{\text{diffusion rate}}$$

where $v$ is a characteristic velocity and $L$ is a characteristic length scale.

| Regime | Condition | Physics |
|--------|-----------|--------|
| **Ideal MHD** | $R_m \gg 1$ | Advection dominates, field frozen to plasma |
| **Resistive MHD** | $R_m \ll 1$ | Diffusion dominates, field slips through plasma |

In this notebook, we study the **resistive limit** ($R_m \ll 1$) where $\mathbf{v} = 0$ and the induction equation reduces to:

$$\frac{\partial \mathbf{B}}{\partial t} = \eta \nabla^2 \mathbf{B}$$

This is the **heat equation** (or diffusion equation) for the magnetic field!

### Analytic Solution: Spreading Gaussian

For a 2D Gaussian initial condition in the x-y plane:

$$B_z(x, y, t=0) = B_{\text{peak}} \exp\left(-\frac{x^2 + y^2}{2\sigma_0^2}\right)$$

The exact solution is a **spreading Gaussian**:

$$B_z(x, y, t) = B_{\text{peak}} \frac{\sigma_0^2}{\sigma_t^2} \exp\left(-\frac{x^2 + y^2}{2\sigma_t^2}\right)$$

where $\sigma_t^2 = \sigma_0^2 + 2Dt$ and $D = \eta/\mu_0$ is the magnetic diffusivity.

Key features:
- **Width grows**: $\sigma(t) = \sqrt{\sigma_0^2 + 2Dt}$
- **Peak decreases**: amplitude $\propto \sigma_0^2/\sigma_t^2$ in 2D
- **Characteristic timescale**: $\tau_{\text{diff}} = \sigma_0^2 / (2D)$

**Important**: The Gaussian is in the x-y plane (uniform in z) to ensure $\nabla \cdot \mathbf{B} = 0$, since $\partial B_z/\partial z = 0$. This is required for the MHD induction equation to correctly reduce to scalar diffusion.

### Physical Intuition

Magnetic diffusion is analogous to heat conduction:

- A localized "hot spot" of magnetic field spreads into surrounding regions
- The field diffuses from high-B to low-B regions
- Higher resistivity ($\eta$) means faster diffusion
- The process is irreversible (entropy increases)

In fusion plasmas, magnetic diffusion is usually slow ($R_m \sim 10^6$ in tokamaks), but it becomes important:
- During magnetic reconnection events
- In resistive instabilities (tearing modes)
- Near plasma boundaries where temperature (and thus conductivity) is lower

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# jax-frc imports
from jax_frc.configurations import MagneticDiffusionConfiguration
from jax_frc.solvers.explicit import RK4Solver

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error, compute_metrics, metrics_summary, animate_overlay

# Plotting style
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === PHYSICS PARAMETERS ===

B_peak = 1.0       # Peak magnetic field [T]
sigma = 0.1        # Initial Gaussian width [m]

# Resistivity (not diffusivity!) - diffusivity D = eta/mu_0
eta = 1.26e-10     # Magnetic resistivity [Ω·m]
MU0 = 1.2566e-6    # Permeability of free space
diffusivity = eta / MU0  # ~1e-4 m²/s

print(f"Resistivity η = {eta:.2e} Ω·m")
print(f"Diffusivity D = η/μ₀ = {diffusivity:.2e} m²/s")

# === GRID PARAMETERS (3D Cartesian) ===

nx = 64            # X resolution
ny = 64            # Y resolution
nz = 1             # Z resolution (pseudo-dimension for 2D-like behavior)
extent = 1.0       # Domain: [-extent, extent] in each direction

print(f"\n3D Grid: {nx} × {ny} × {nz} (x × y × z)")
print(f"Domain: [{-extent}, {extent}]³")
print(f"Thin z-direction makes this effectively 2D in x-y plane")

# === TIME PARAMETERS ===

# Diffusion timescale
tau_diff = sigma**2 / (2 * diffusivity)
print(f"\nDiffusion timescale: τ_diff = {tau_diff:.2f} s")

t_end = 0.3 * tau_diff    # Run for 0.3 diffusion times (3X longer)

# CFL constraint for diffusion
dx = 2 * extent / nx
dy = 2 * extent / ny
dx_min = min(dx, dy)
dt_max = 0.25 * dx_min**2 / diffusivity
dt = dt_max * 0.5  # Safety factor

print(f"Simulation time: t_end = {t_end:.2f} s (0.3 τ_diff)")
print(f"Timestep: dt = {dt:.4f} s (CFL: {dt_max:.4f} s)")
print(f"Expected steps: ~{int(t_end/dt)}")

# === MODELS TO TEST ===
MODELS_TO_TEST = ["resistive_mhd", "extended_mhd"]
MODEL_STYLES = {
    "resistive_mhd": {"label": "ResistiveMHD", "color": "blue", "linestyle": "-"},
    "extended_mhd": {"label": "ExtendedMHD", "color": "orange", "linestyle": "-"},
}
print(f"\nModels to test: {MODELS_TO_TEST}")

In [ ]:
# Create configurations for both models
configs = {}
models = {}

for model_type in MODELS_TO_TEST:
    config = MagneticDiffusionConfiguration(
        B_peak=B_peak,
        sigma=sigma,
        eta=eta,
        nx=nx,
        ny=ny,
        nz=nz,
        extent=extent,
        model_type=model_type,
    )
    configs[model_type] = config
    models[model_type] = config.build_model()

# Use first config for geometry (same for all models)
geometry = configs["resistive_mhd"].build_geometry()
initial_state = configs["resistive_mhd"].build_initial_state(geometry)
solver = RK4Solver()

print(f"Grid: {geometry.nx} × {geometry.ny} × {geometry.nz} (x × y × z)")
print(f"Domain: x ∈ [{geometry.x_min:.1f}, {geometry.x_max:.1f}]")
print(f"        y ∈ [{geometry.y_min:.1f}, {geometry.y_max:.1f}]")
print(f"        z ∈ [{geometry.z_min:.1f}, {geometry.z_max:.1f}]")
print(f"Resolution: Δx = {geometry.dx:.4f}, Δy = {geometry.dy:.4f}, Δz = {geometry.dz:.4f}")
print(f"\nModels:")
for model_type in MODELS_TO_TEST:
    print(f"  - {model_type}: {type(models[model_type]).__name__}")
print(f"\nMagnetic Reynolds number: Rm = {configs['resistive_mhd'].magnetic_reynolds_number():.2e} (should be << 1)")

### Visualize Initial Condition

In [ ]:
# Get 3D grid coordinates
x_3d = np.array(geometry.x_grid)
y_3d = np.array(geometry.y_grid)
z_3d = np.array(geometry.z_grid)

# Initial B_z profile - take a slice at z=0 (middle of thin z dimension)
z_mid_idx = geometry.nz // 2
Bz_init_2d = np.array(initial_state.B[:, :, z_mid_idx, 2])  # x-y slice

# 2D coordinates for plotting (x-y slice)
x_2d = x_3d[:, :, z_mid_idx]
y_2d = y_3d[:, :, z_mid_idx]

# Create figure with 2D contour and cross-sections
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 2D contour plot (x-y slice)
im = axes[0].contourf(y_2d, x_2d, Bz_init_2d, levels=20, cmap='RdBu_r')
axes[0].set_xlabel('y [m]', fontsize=12)
axes[0].set_ylabel('x [m]', fontsize=12)
axes[0].set_title('Initial $B_z$ (x-y slice at z=0)', fontsize=14)
plt.colorbar(im, ax=axes[0], label='$B_z$ [T]')
# Mark center
axes[0].plot(0, 0, 'k+', markersize=15, markeredgewidth=2)

# X slice at y=0
y_mid_idx = geometry.ny // 2
x_1d = x_2d[:, y_mid_idx]
Bz_x = Bz_init_2d[:, y_mid_idx]
axes[1].plot(x_1d, Bz_x, 'b-', linewidth=2)
axes[1].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('x [m]', fontsize=12)
axes[1].set_ylabel('$B_z$ [T]', fontsize=12)
axes[1].set_title('X slice at y=0', fontsize=14)
axes[1].grid(True, alpha=0.3)

# Y slice at x=0
x_mid_idx = geometry.nx // 2
y_1d = y_2d[x_mid_idx, :]
Bz_y = Bz_init_2d[x_mid_idx, :]
axes[2].plot(y_1d, Bz_y, 'b-', linewidth=2)
axes[2].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('y [m]', fontsize=12)
axes[2].set_ylabel('$B_z$ [T]', fontsize=12)
axes[2].set_title('Y slice at x=0', fontsize=14)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Peak B_z = {np.max(Bz_init_2d):.3f} T at origin")
print(f"Initial width σ₀ = {sigma} m")

---
## 3. Run Simulation

We evolve the Gaussian profile forward in time using both ResistiveMHD and ExtendedMHD models. The magnetic field should spread isotropically according to the analytic solution, and both models should produce identical results.

In [ ]:
# Time stepping setup
n_steps = int(t_end / dt)
save_interval = max(1, n_steps // 50)  # ~50 snapshots

print(f"Running {n_steps} steps with dt = {dt:.2e}")
print(f"Saving every {save_interval} steps")

In [ ]:
# Run simulation for each model
results = {}

for model_type in MODELS_TO_TEST:
    print(f"Running {model_type}...")
    model = models[model_type]
    state = initial_state
    
    # Define scan step for this model
    @jax.jit
    def scan_step(state, _):
        new_state = solver.step(state, dt, model, geometry)
        return new_state, new_state.B[:, :, z_mid_idx, 2]
    
    # Run simulation
    final_state, Bz_history = lax.scan(scan_step, state, None, length=n_steps)
    
    # Store results
    Bz_history = np.array(Bz_history)[::save_interval]
    Bz_final_2d = np.array(final_state.B[:, :, z_mid_idx, 2])
    
    results[model_type] = {
        "final_state": final_state,
        "Bz_final_2d": Bz_final_2d,
        "Bz_history": Bz_history,
    }
    print(f"  Completed")

print(f"\nSimulation complete for all models")

---
## 4. Compare with Analytic Solution

In [ ]:
# Compute 2D analytic solution at final time (x-y plane)
t_final = n_steps * dt
config = configs["resistive_mhd"]  # Use any config for analytic solution
Bz_analytic_2d = np.array(config.analytic_solution_2d(
    jnp.array(x_2d), jnp.array(y_2d), t_final
))

# Expected width at final time
sigma_final = np.sqrt(sigma**2 + 2 * diffusivity * t_final)
print(f"Initial width: σ₀ = {sigma:.4f} m")
print(f"Final width:   σ(t) = {sigma_final:.4f} m")
print(f"Width increase: {(sigma_final/sigma - 1)*100:.1f}%")
print(f"\nPeak decay: σ₀²/σ_t² = {(sigma/sigma_final)**2:.3f} (2D scaling)")

In [ ]:
# Combined comparison plots showing both models and analytic
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Top row: 2D contours for Initial, ResistiveMHD, ExtendedMHD
vmax = B_peak
levels = np.linspace(0, vmax, 21)

im0 = axes[0, 0].contourf(y_2d, x_2d, Bz_init_2d, levels=levels, cmap='RdBu_r')
axes[0, 0].set_title('Initial $B_z$', fontsize=12)
axes[0, 0].set_ylabel('x [m]')
plt.colorbar(im0, ax=axes[0, 0])

im1 = axes[0, 1].contourf(y_2d, x_2d, results["resistive_mhd"]["Bz_final_2d"], levels=levels, cmap='RdBu_r')
axes[0, 1].set_title(f'ResistiveMHD at t={t_final:.2f}s', fontsize=12)
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[0, 2].contourf(y_2d, x_2d, results["extended_mhd"]["Bz_final_2d"], levels=levels, cmap='RdBu_r')
axes[0, 2].set_title(f'ExtendedMHD at t={t_final:.2f}s', fontsize=12)
plt.colorbar(im2, ax=axes[0, 2])

# Bottom row: Combined slices showing all three (ResistiveMHD, ExtendedMHD, Analytic)

# X slice at y=0 - all three on same plot
axes[1, 0].plot(x_1d, Bz_init_2d[:, y_mid_idx], 'gray', linestyle=':', label='Initial')
axes[1, 0].plot(x_1d, Bz_analytic_2d[:, y_mid_idx], 'k--', linewidth=2, label='Analytic')
for model_type in MODELS_TO_TEST:
    style = MODEL_STYLES[model_type]
    Bz_sim = results[model_type]["Bz_final_2d"][:, y_mid_idx]
    axes[1, 0].plot(x_1d, Bz_sim, color=style["color"], linestyle=style["linestyle"],
                    linewidth=1.5, label=style["label"])
axes[1, 0].set_xlabel('x [m]')
axes[1, 0].set_ylabel('$B_z$ [T]')
axes[1, 0].set_title('X slice at y=0')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Y slice at x=0 - all three on same plot
axes[1, 1].plot(y_1d, Bz_init_2d[x_mid_idx, :], 'gray', linestyle=':', label='Initial')
axes[1, 1].plot(y_1d, Bz_analytic_2d[x_mid_idx, :], 'k--', linewidth=2, label='Analytic')
for model_type in MODELS_TO_TEST:
    style = MODEL_STYLES[model_type]
    Bz_sim = results[model_type]["Bz_final_2d"][x_mid_idx, :]
    axes[1, 1].plot(y_1d, Bz_sim, color=style["color"], linestyle=style["linestyle"],
                    linewidth=1.5, label=style["label"])
axes[1, 1].set_xlabel('y [m]')
axes[1, 1].set_ylabel('$B_z$ [T]')
axes[1, 1].set_title('Y slice at x=0')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Error plot - both models
axes[1, 2].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
for model_type in MODELS_TO_TEST:
    style = MODEL_STYLES[model_type]
    error = results[model_type]["Bz_final_2d"][x_mid_idx, :] - Bz_analytic_2d[x_mid_idx, :]
    axes[1, 2].plot(y_1d, error, color=style["color"], linestyle=style["linestyle"],
                    linewidth=1.5, label=style["label"])
axes[1, 2].set_xlabel('y [m]')
axes[1, 2].set_ylabel('Error [T]')
axes[1, 2].set_title('Error (Simulation - Analytic)')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Error distribution (use shared utility on flattened arrays)
fig, axes = plot_error(
    np.arange(Bz_final_2d.size), Bz_final_2d.ravel(), Bz_analytic_2d.ravel(),
    xlabel='Grid point index',
    title='Error Analysis (flattened)'
)
plt.show()

In [ ]:
# Compute validation metrics for each model
print("Validation Metrics:")
print("=" * 60)
print(f"{'Model':<15} {'L2 Error':<12} {'Linf Error':<12} {'Status'}")
print("-" * 60)

thresholds = {
    'l2_error': 0.01,       # 1% relative L2 error
    'linf_error': 0.02,     # 2% max error
}

all_pass = True
for model_type in MODELS_TO_TEST:
    Bz_sim = results[model_type]["Bz_final_2d"]
    metrics = compute_metrics(Bz_sim.ravel(), Bz_analytic_2d.ravel())
    
    l2_pass = metrics['l2_error'] <= thresholds['l2_error']
    linf_pass = metrics['linf_error'] <= thresholds['linf_error']
    model_pass = l2_pass and linf_pass
    all_pass = all_pass and model_pass
    
    status = '✓ PASS' if model_pass else '✗ FAIL'
    print(f"{model_type:<15} {metrics['l2_error']:<12.4e} {metrics['linf_error']:<12.4e} {status}")
    
    results[model_type]["metrics"] = metrics

print("=" * 60)
print(f"Overall: {'✓ PASS' if all_pass else '✗ FAIL'}")
print(f"\nThresholds: L2 < {thresholds['l2_error']:.0%}, Linf < {thresholds['linf_error']:.0%}")

---
## 5. Time Evolution Animation

In [ ]:
# 2D animation of spreading Gaussian - use ResistiveMHD for animation
from matplotlib.animation import FuncAnimation

Bz_history_anim = results["resistive_mhd"]["Bz_history"]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Initialize plots
levels = np.linspace(0, B_peak, 21)

def animate(frame):
    t = times[frame]
    
    # Clear and replot (contourf doesn't support set_array)
    axes[0].clear()
    axes[0].contourf(y_2d, x_2d, Bz_history_anim[frame], levels=levels, cmap='RdBu_r')
    axes[0].set_xlabel('y [m]')
    axes[0].set_ylabel('x [m]')
    axes[0].set_title(f'Numerical (t={t:.3f}s)')
    
    axes[1].clear()
    Bz_ana_t = np.array(config.analytic_solution_2d(jnp.array(x_2d), jnp.array(y_2d), t))
    axes[1].contourf(y_2d, x_2d, Bz_ana_t, levels=levels, cmap='RdBu_r')
    axes[1].set_xlabel('y [m]')
    axes[1].set_ylabel('x [m]')
    axes[1].set_title(f'Analytic (t={t:.3f}s)')
    
    return []

anim = FuncAnimation(fig, animate, frames=len(times), interval=200, blit=False)
plt.close()

HTML(anim.to_jshtml())

---
## 6. Interactive Exploration

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def explore_time(time_idx):
    """Show solution at a specific snapshot comparing both models."""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    t = times[time_idx]
    Bz_ana = np.array(config.analytic_solution_2d(jnp.array(x_2d), jnp.array(y_2d), t))
    
    # Top row: 2D contours
    levels = np.linspace(0, B_peak, 21)
    
    im0 = axes[0, 0].contourf(y_2d, x_2d, Bz_ana, levels=levels, cmap='RdBu_r')
    axes[0, 0].set_xlabel('y [m]')
    axes[0, 0].set_ylabel('x [m]')
    axes[0, 0].set_title(f'Analytic at t = {t:.4f} s')
    plt.colorbar(im0, ax=axes[0, 0])
    
    for i, model_type in enumerate(MODELS_TO_TEST):
        Bz_num = results[model_type]["Bz_history"][time_idx]
        im = axes[0, i+1].contourf(y_2d, x_2d, Bz_num, levels=levels, cmap='RdBu_r')
        axes[0, i+1].set_xlabel('y [m]')
        axes[0, i+1].set_ylabel('x [m]')
        axes[0, i+1].set_title(f'{MODEL_STYLES[model_type]["label"]} at t = {t:.4f} s')
        plt.colorbar(im, ax=axes[0, i+1])
    
    # Bottom left: X slice at y=0 - all three
    axes[1, 0].plot(x_1d, Bz_init_2d[:, y_mid_idx], 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[1, 0].plot(x_1d, Bz_ana[:, y_mid_idx], 'k--', linewidth=2, label='Analytic')
    for model_type in MODELS_TO_TEST:
        style = MODEL_STYLES[model_type]
        Bz_num = results[model_type]["Bz_history"][time_idx]
        axes[1, 0].plot(x_1d, Bz_num[:, y_mid_idx], color=style["color"], 
                        linestyle=style["linestyle"], linewidth=1.5, label=style["label"])
    axes[1, 0].set_xlabel('x [m]')
    axes[1, 0].set_ylabel('$B_z$ [T]')
    axes[1, 0].set_title('X slice at y=0')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim(0, B_peak * 1.1)
    
    # Width indicator
    sigma_t = np.sqrt(sigma**2 + 2 * diffusivity * t)
    axes[1, 0].axvline(-sigma_t, color='green', linestyle='--', alpha=0.5)
    axes[1, 0].axvline(sigma_t, color='green', linestyle='--', alpha=0.5)
    
    # Bottom middle: Y slice at x=0 - all three
    axes[1, 1].plot(y_1d, Bz_init_2d[x_mid_idx, :], 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[1, 1].plot(y_1d, Bz_ana[x_mid_idx, :], 'k--', linewidth=2, label='Analytic')
    for model_type in MODELS_TO_TEST:
        style = MODEL_STYLES[model_type]
        Bz_num = results[model_type]["Bz_history"][time_idx]
        axes[1, 1].plot(y_1d, Bz_num[x_mid_idx, :], color=style["color"],
                        linestyle=style["linestyle"], linewidth=1.5, label=style["label"])
    axes[1, 1].set_xlabel('y [m]')
    axes[1, 1].set_ylabel('$B_z$ [T]')
    axes[1, 1].set_title('Y slice at x=0')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_ylim(0, B_peak * 1.1)
    
    # Bottom right: Error comparison
    axes[1, 2].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    for model_type in MODELS_TO_TEST:
        style = MODEL_STYLES[model_type]
        Bz_num = results[model_type]["Bz_history"][time_idx]
        error = Bz_num[x_mid_idx, :] - Bz_ana[x_mid_idx, :]
        axes[1, 2].plot(y_1d, error, color=style["color"],
                        linestyle=style["linestyle"], linewidth=1.5, label=style["label"])
    axes[1, 2].set_xlabel('y [m]')
    axes[1, 2].set_ylabel('Error [T]')
    axes[1, 2].set_title('Error (Simulation - Analytic)')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics for both models
    print(f"Current width σ(t) = {sigma_t:.4f} m")
    print(f"\n{'Model':<15} {'L2 Error':<12} {'Linf Error':<12}")
    print("-" * 40)
    for model_type in MODELS_TO_TEST:
        Bz_num = results[model_type]["Bz_history"][time_idx]
        m = compute_metrics(Bz_num.ravel(), Bz_ana.ravel())
        print(f"{model_type:<15} {m['l2_error']:<12.4e} {m['linf_error']:<12.4e}")

# Time slider
time_slider = widgets.IntSlider(
    value=0, min=0, max=len(times)-1, step=1,
    description='Snapshot:',
    continuous_update=False
)

ui = widgets.VBox([
    widgets.HTML('<h4>Time Evolution Explorer - Multi-Model Comparison</h4>'),
    widgets.HTML('<p>Drag the slider to compare ResistiveMHD and ExtendedMHD at different times.</p>'),
    time_slider,
])

interactive_output = widgets.interactive_output(explore_time, {'time_idx': time_slider})

display(ui, interactive_output)

---
## Summary

This notebook demonstrated **magnetic field diffusion** using 3D Cartesian coordinates with **two MHD models**:

1. **Physics**: In the resistive limit ($R_m \ll 1$), the magnetic induction equation reduces to a diffusion equation. Magnetic field spreads through the plasma like heat through a conductor.

2. **Analytic solution**: A Gaussian initial condition evolves as:
   - Width: $\sigma(t) = \sqrt{\sigma_0^2 + 2Dt}$ (isotropic spreading)
   - Peak: $B_{\text{peak}}(t) = B_0 \cdot \sigma_0^2/\sigma_t^2$ (2D amplitude decay)

3. **Model comparison**: Both **ResistiveMHD** and **ExtendedMHD** produce identical results for pure diffusion, confirming they implement the same physics when Hall and electron pressure terms are disabled.

4. **3D Cartesian coordinates**: The simulation uses native Cartesian (x, y, z) coordinates. For 2D-like behavior, we use a thin z dimension (nz=1) with periodic boundaries. The Gaussian is in the x-y plane to ensure $\nabla \cdot \mathbf{B} = 0$.

5. **Validation**: Both models match the analytic solution with < 1% L2 error, confirming correct implementation of resistive diffusion.

### Physical Significance

Magnetic diffusion is important in:
- **Magnetic reconnection**: Field lines can break and reconnect when diffusion is significant
- **Resistive instabilities**: Tearing modes, resistive wall modes
- **Edge physics**: Lower temperature means higher resistivity and faster diffusion

### Try Next

- Increase resolution (`nx`, `ny`) to reduce numerical error
- Increase `nz` to see full 3D diffusion
- Compare with the **frozen flux** notebook to see the opposite limit ($R_m \gg 1$)